In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tqdm.notebook as tqdm
import gtsam
from gtsam.symbol_shorthand import P, X
from art_skills import ExpressionFactorPoint2, ExpressionDouble, ExpressionPoint2, DebugExpressionPoint2, DebugExpressionFactorPoint2
from art_skills import SlnStrokeExpression2
import loader
import initialize_utils
import utils
from utils import Double_, Point2_, Time
from fit_types import OptimizationLoggingParams, Solution
import plotting2
import sln_fit

%load_ext autoreload
%autoreload 2
%aimport -numpy
%aimport -matplotlib
%aimport -tqdm
%aimport -gtsam

# Setup

In [ ]:
# Constants
noise_model = gtsam.noiseModel.Isotropic.Sigma(2, 1.0)
# Data
letter_data = loader.load_segments('A', index=None)
trajectory_data = letter_data[0]
stroke_data = trajectory_data[1]

In [ ]:
# Debugging
stroke = SlnStrokeExpression2(P(0))
def create_debug_expression_cb(stroke, t, x, y, only_on_failure=True):
    def debug_expression_cb(values, ret):
        # if any(np.isnan(ret)):
        #     print(f'(t, x, y) = ({t:.3f}, {x:.2f}, {y:.2f}), pred = {ret}, P = {values.atVector(P(0)).tolist()}, X = {values.atVector(X(0)).tolist()}')
        if any (np.isnan(stroke.pos(t, x0=values.atPoint2(X(0)), values=values))):
            print(stroke.pos(Double_(t), values.atPoint2(X(0)), 1e-2).value(values))
    return debug_expression_cb

# Fitting - Minimum effort demos

In [ ]:
plotting2.plot_stroke(None, sln_fit.fit_stroke(stroke_data))

In [ ]:
fix, ax = plt.subplots(figsize=(10, 10))
plotting2.plot_trajectory(ax, sln_fit.fit_trajectory(trajectory_data), disp_params=False)

In [ ]:
with Time('fit letter'):
    sol = sln_fit.fit_letter(letter_data)
with Time('plot'):
    plotting2.plot_letter(None, sol)

In [ ]:
params = sln_fit.FitStrokeParams(
    lm_params=utils.create_params(absoluteErrorTol=0, relativeErrorTol=0))
fig, axes = plt.subplots(1, 5, figsize=(25, 5))
all_sols = []
with Time('Fit letters'):
    for letter in 'ABCDE':
        letter_data = loader.load_segments(letter, index=None)
        all_sols.append(sln_fit.fit_letter(letter_data, fit_params=params))
        break
with Time('Plotting'):
    for sol, ax in zip(all_sols, axes):
        plotting2.plot_letter(ax, sol)

# Experimentation

## Weighting end-points higher

Observe in the results of the next cell that:
* W/ edge weighting looks better than w/o edge-weighting
* Optimizing w/o edge weighting, but initializing with the solution from edge-weighting looks very similar to w/ edge weighting
* The MSE (mean-squared-error) is actually lower w/o edge weighting! (even though it looks worse)

In [ ]:
_, axes = plt.subplots(1,3,figsize=(10, 5))
sol = sln_fit.fit_stroke(stroke_data,
                         fit_params=sln_fit.FitStrokeParams(noise_model_connecting=None))
plotting2.plot_stroke(axes[0], sol)
axes[0].set_title('Default (without end-point constraints)')
sol = sln_fit.fit_stroke(stroke_data,
                         fit_params=sln_fit.FitStrokeParams(
                             noise_model_connecting=gtsam.noiseModel.Isotropic.Sigma(2, 1e-2)))
plotting2.plot_stroke(axes[1], sol)
axes[1].set_title('With end-point constraints')
sol = sln_fit.fit_stroke(stroke_data,
                         fit_params=sln_fit.FitStrokeParams(noise_model_connecting=None),
                         initial_guess=sol)
plotting2.plot_stroke(axes[2], sol)
axes[2].set_title('Without end-point constraints,\nbut using initial guess w/ end-point constraints')

## Random Restarts
Question: what is the true global minimum?

Let's try random-restarts:
* Optimize using a random initialization
* Repeat 100 times and take the best result

In [ ]:
errors_and_sols = []
for i in range(100):
    init_guess = utils.ValuesFromDict({P(0): np.random.randn(6), X(0): np.random.randn(2)})
    sol = sln_fit.fit_stroke(stroke_data,
                             fit_params=sln_fit.FitStrokeParams(noise_model_connecting=None,
                                                                lm_params=utils.create_params(
                                                                    absoluteErrorTol=0,
                                                                    maxIterations=500)),
                             initial_guess=init_guess)
    errors_and_sols.append((sln_fit.StrokeUtils.MSE(sol), i, sol)) # add `i` to tuple as tiebreaker

In [ ]:
_, axes = plt.subplots(1, len(errors_and_sols) // 10, figsize=(40, 10))
for ax, (_, _, sol) in zip(axes, sorted(errors_and_sols)[::10]):
    plotting2.plot_stroke(ax, sol)
    ax.get_legend().remove()

In [ ]:
mses = [mse for mse, _, _ in errors_and_sols]
bins = 10**np.linspace(np.log10(min(mses)), np.log10(max(mses)), 20)
plt.xscale('log')
plt.hist(mses, bins=bins);
plt.xlabel('MSE')
plt.ylabel('Number of trials')
plt.title('Histogram of Final regression errors')

In [ ]:
for mse, _, sol in sorted(errors_and_sols):
    print('MSE: {:.5e}  Params: {:5.2f}  {:5.2f}  {:5.2f}  {:5.2f}  {:5.2f}  {:5.2f}'.format(mse, *sol['params']))
sum(1 for mse, _, _ in errors_and_sols if mse < 1.1e-4)